# TF-IDF
## 단어 빈도의 문제
- 단어 문서 행렬에서 한 문서는 단어 빈도를 통해 표현
- <b>가정</b>: 단어 빈도가 비슷한 문서 = 의미가 비슷한 문서
- 실제로는 정확하지 않음 (예: 특정 단어가 무의미하게 반복되는 경우)
- 단어 빈도에 적절한 가중치를 주어 극복

## 단어 빈도의 가중치들
- binary: 단어의 빈도를 0 또는 1로만 표시 (1 이상도 1)
 - 무의미한 반복을 막을 수 있지만 극단적
 - 구체적으로 몇 번 나왔는지 안 중요한 경우
- 로그 함수를 적용
 - 쓸데없이 많은 반복의 효과를 감소시킴
 - 많이 나오는 것의 효과를 줄이고 싶지만, binary처럼 아예 무시하고 싶지는 않을 때
- 문서의 총 단어 빈도로 나눔 (글의 길이로 보정)
 - 문서의 길이가 제각각일 때
- 가장 많은 단어의 빈도로 나눔

## 문서 빈도 (Document Frequency)
- 문서빈도(df): 각 단어가 등장한(tf > 0) 문서의 수
- 역문서빈도(idf): 총 문서 수를 df로 나눈 값
 - 단어가 얼마나 드물게 등장하는지 알 수 있고, 다른 문서와의 차이를 보여줌
 - 단어에 대한 중요도로 사용됨
- 여러 문서에 자주 나오면 df ↑, idf ↓
- 문서 간의 차이가 중요한 상황에서는 idf가 높은 단어가 좋은 단어

||맛집|시청|밥|
|:---|:---:|:---:|:---:|
|①|3|1|1|
|②|2|0|2|
|③|0|0|1|
|④|0|0|3|
|**df**|2|1|4|
|**idf**|2|4|1|

## TF-IDF
```
똑같이 자주 나오는 단어라고 해도, 모든 문서에 고르게 자주 나오는 단어는 특별한 의미가 없는 경우가 많다. 핸드폰 리뷰에서는 당연히 핸드폰이라는 단어가 자주 쓰이게지만, 어차피 모든 리뷰에 많이 나온다면 리뷰들 사이의 차이를 분석할 때는 크게 도움이 되지 않는다. TF-IDF는 이러한 측면을 고려하여 단어의 빈도를 보정하는 방법이다.
```
- 단어 빈도(TF)와 역문서빈도(IDF)를 곱한 값
 - TF: 단어(**T**erm)의 등장 빈도(**F**requency)
 - IDF: 특정 단어가 등장한 문서(**D**ocument)의 빈도(**F**requency)의 역수(**I**nverse)
- 상대적으로 적은 문서에 나오면서 특정 문서에 자주 나온 단어
- TDM에 가중치를 주는 대표적 방법
 - 단순히 단어가 많거나 적게 사용되는 정도를 보정하는 것이 아니라 단어의 중요성을 보정해줌
- 다른 방법들도 사용 가능. 분석 목적에 맞게 사용.

---
# TF-IDF 실습
## 데이터 불러오기

In [1]:
import pandas as pd
df = pd.read_excel('data/imdb.xlsx', index_col=0)
df.head()

,review,sentiment
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1


## TF-IDF

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer

#### `TfidfVectorizer`의 옵션
- `stop_words`: 분석에서 제외할 불용어를 설정하는 옵션
 - `stop_words='english'`: 영어는 관사, 전치사 등을 제외
- `max_features`: 단어문서행렬에 포함시킬 최대(max)의 단어(feature) 수
 - `max_features=500`: 빈도 순으로 최대 500 단어까지 포함

In [3]:
# 초기화
tfidf = TfidfVectorizer(max_features=500, stop_words='english')

In [4]:
# 단어문서행렬 만들기
tdm = tfidf.fit_transform(df['review'])

# 748개 문서의 500개 피처가 결과로 반환
tdm

<748x500 sparse matrix of type '<class 'numpy.float64'>'
	with 3433 stored elements in Compressed Sparse Row format>

## 단어 빈도순 정렬

In [5]:
word_count = pd.DataFrame({
    '단어': tfidf.get_feature_names(),
    'tf-idf': tdm.sum(axis=0).flat
})
word_count.sort_values('tf-idf', ascending=False).head()

,단어,tf-idf
281,movie,43.842614
155,film,40.296277
31,bad,25.212655
227,just,20.498951
181,good,18.534937


대체로 `CountVectorizer`와 크게 차이가 나지는 않지만, 수치 자체가 달라짐

## 기존의 TDM에서 변환하기: `TfidfTransformer`
`CountVectorizer`로 만든 단어 문서 행렬을 변환

In [6]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

In [7]:
cv = CountVectorizer(max_features=500, stop_words='english')

In [8]:
tdm2 = cv.fit_transform(df['review'])

In [9]:
trans = TfidfTransformer()

In [10]:
tdm3 = trans.fit_transform(tdm2)

In [11]:
wc2 = pd.DataFrame({
    '단어': cv.get_feature_names(),
    'tf-idf': tdm3.sum(axis=0).flat
})
wc2.sort_values('tf-idf', ascending=False).head()

,단어,tf-idf
281,movie,43.842614
155,film,40.296277
31,bad,25.212655
227,just,20.498951
181,good,18.534937


## 두 가지 방법으로 만든 TDM의 비교

In [12]:
import numpy as np

In [13]:
# .allclose로 모든 값이 가까운지 확인
# .A로 압축 풀기
np.allclose(tdm.A, tdm3.A)

True